In [ ]:
# default_exp weights
%load_ext autoreload
%autoreload 2
# from nbdev.export2html import nbdev_build_docs
# nbdev_build_docs()

# Weights
> Load weighted data, combine with photon data

In [ ]:
#export
import os,  pickle, healpy
import numpy as np
from light_curves.config import Config, FileConfiguration, PointSource
from light_curves.load_gti import get_gti
from light_curves.photon_data import get_photon_data

config = None
files = None

Set up photon data for tests

In [ ]:
config = Config()
files = FileConfiguration()
source = PointSource('Geminga')

if files.valid:
    print(f'Loading photon data for using source {source.name} to test adding weights')
    photon_data = get_photon_data(config, files,  source )
else:
    print('Not testing since no files.')

Loading photon data for using source Geminga to test adding weights
Processing 11 GTI files ...  11 files, 63635 intervals with 3,322 days live time
	GTI MJD range: 54682.66-58698.08, good fraction 0.83 
Loading  132 months from Arrow dataset /home/burnett/data/dataset
....................................................................................................................................
	Selected 1313726 photons within 5 deg of  (195.13,4.27)
	Energies: 100.0-1000000 MeV
	Dates:    2008-08-04 15:46 - 2019-08-03 01:17
	MJD  :    54682.7          - 58698.1         


In [ ]:
#export
def _load_weights(filename, ):
    """Load the weight informaton 

    filename: pickled dict with map info
 
    """     
    # load a pickle containing weights, generated by pointlike
    assert os.path.exists(filename),f'File {filename} not found.'
    with open(filename, 'rb') as file:
        wtd = pickle.load(file, encoding='latin1')
    assert type(wtd)==dict, 'Expect a dictionary'
    test_elements = 'energy_bins pixels weights nside model_name radius order roi_name'.split()
    assert np.all([x in wtd.keys() for x in test_elements]),f'Dict missing one of the keys {test_elements}'
    if config.verbose>0:
        print(f'Load weights from file {os.path.realpath(filename)}')
        pos = wtd['source_lb']
        print(f'\tFound: {wtd["source_name"]} at ({pos[0]:.2f}, {pos[1]:.2f})')
    # extract pixel ids and nside used
    wt_pix   = wtd['pixels']
    nside_wt = wtd['nside']

    # merge the weights into a table, with default nans
    # indexing is band id rows by weight pixel columns
    # append one empty column for photons not in a weight pixel
    # calculated weights are in a dict with band id keys        
    wts = np.full((32, len(wt_pix)+1), np.nan, dtype=np.float32)    
    weight_dict = wtd['weights']
    for k in weight_dict.keys():
        t = weight_dict[k]
        if len(t.shape)==2:
            t = t.T[0] #???
        wts[k,:-1] = t   
    return wts , wt_pix , nside_wt 

In [ ]:

weight_file =  os.path.join(files.weights, source.name+'_weights.pkl')
assert os.path.exists( weight_file ), 'Failed check for weights'
wts, wt_pix, nside_wt = _load_weights(weight_file)


Load weights from file /mnt/c/users/thbur/OneDrive/fermi/weight_files/Geminga_weights.pkl
	Found: PSR J0633+1746 at (195.14, 4.27)


In [ ]:
#export
def _add_weights(wts, wt_pix, nside_wt, photon_data):
    # get the photon pixel ids, convert to NEST (if not already) and right shift them 
  
    if not config.nest:
        # data are RING
        photon_pix = healpy.ring2nest(config.nside, photon_data.pixel.values)
    else:
        photon_pix = photon_data.pixel.values
    to_shift = 2*int(np.log2(config.nside/nside_wt)); 
    shifted_pix =   np.right_shift(photon_pix, to_shift)
    bad = np.logical_not(np.isin(shifted_pix, wt_pix)) 
    if config.verbose>0:
        print(f'\tApplyng weights: {sum(bad)} / {len(bad)} photon pixels are outside weight region')
    if sum(bad)==len(bad):
        a = np.array(healpy.pix2ang(nside_wt, wt_pix, nest=True, lonlat=True)).mean(axis=1).round(1)
        b = np.array(healpy.pix2ang(nside_wt, shifted_pix, nest=True, lonlat=True)).mean(axis=1).round(1)
        
        raise Exception(f'There was no overlap of the photon data at {b} and the weights at {a}')
    shifted_pix[bad] = 12*nside_wt**2 # set index to be beyond pixel indices

    # find indices with search and add a "weights" column
    # (expect that wt_pix are NEST ordering and sorted) 
    weight_index = np.searchsorted(wt_pix,shifted_pix)
    band_index = np.fmin(31, photon_data.band.values) #all above 1 TeV into last bin

    # final grand lookup -- isn't numpy wonderful!
    photon_data.loc[:,'weight'] = wts[tuple([band_index, weight_index])] 
    if config.verbose>0:
        print(f'\t{sum(np.isnan(photon_data.weight.values))} weights set to NaN')
      

For each photon, add a weight appropriate for its energy and position

In [ ]:
_add_weights(wts, wt_pix, nside_wt , photon_data)

	Applyng weights: 240 / 1313726 photon pixels are outside weight region
	233109 weights set to NaN


The photon data now has a weight columnn: 

In [ ]:
#hide
photon_data.head()

,band,time,pixel,radius,weight
0,6,54682.657022,6738278,0.698381,0.782784
1,3,54682.657934,6761152,2.498099,NaN
2,4,54682.658637,6739138,0.290310,0.955831
3,1,54682.658760,6714890,3.276757,NaN
4,11,54682.659099,6736721,4.899003,0.002628


In [ ]:
#export
def add_weights(xconfig, xfiles, photon_data, source):
    """ add weights for the soruce to the photons data
    """
    global config, files
    config = xconfig
    files = xfiles
    
    weight_file =  os.path.join(files.weights, source.name+'_weights.pkl')
    assert os.path.exists(weight_file)
    
    wts, wt_pix, nside_wt = _load_weights(weight_file)
    _add_weights(wts, wt_pix, nside_wt, photon_data)
    

In [ ]:
# add_weights(config, files, photon_data, source)